In [ ]:
import networkx as nx
import numpy as np

### Specifying parameters from Tucker model

In [ ]:
n_pop = 18700

# Isoboxes
n_isoboxes = 810
pop_isoboxes = 8100
pop_per_isobox = 10

# Tents
n_tents = 2650
pop_tents = 10600
pop_per_tent = 4

# Others 
n_bathrooms = 144
n_ethnic_groups = 8

In [ ]:
population = list(range(n_pop))

In [ ]:
g = nx.Graph()

In [ ]:
for i in range(pop_isoboxes):
    g.add_node(i)

### Assigning population to isoboxes

In [ ]:
max_pop_per_isobox = poisson.rvs(mu=10, size=810)

In [ ]:
max_pop_per_isobox[:10]

In [ ]:
g = nx.Graph()
iso_count = np.zeros(shape=(810))
nodes_per_isobox = [[] for i in range(810)]

for node in range(pop_isoboxes):
    g.add_node(node)
    
    iso_num = np.random.choice(range(810))
    while iso_count[iso_num] == max_pop_per_isobox[iso_num]:
        iso_num = np.random.choice(range(810))
    
    g.nodes[node]["isobox_num"] = iso_num
    iso_count[iso_num] += 1
    
    nodes_per_isobox[iso_num].append(node)


### Add connections between people from the same isobox

In [ ]:
import itertools

In [ ]:
for node_list in nodes_per_isobox:
    edge_list = [tup for tup in list(itertools.product(node_list, repeat=2)) if tup[0] != tup[1]]
    g.add_edges_from(edge_list)

### Create FRIENDSHIP GRID

In [ ]:
width = 29
height = 28

iso_grid = np.zeros(shape=(width, height)).astype(int)

iso_n = 0

for i in range(width):
    for j in range(height):
        iso_grid[i][j] = iso_n
        iso_n += 1


iso_grid

In [ ]:
# At most 8 isobox neghbors if degree = 1, 24 if degree = 2, neighbors= (2*degree+1)^2 - 1
# Access neighbors through formula of degree and grid, connect them using node properties ethnicity and gender
n_neighbors = ((2*iso_proximity+1)**2)-1
n_neighbors

In [ ]:
def get_iso_neighbors(grid, iso_coords, proximity):
    """ Given a grid of isobox numbers, returns the closest proximity neighbors to the given isobox
    
        params:
        - Grid: 2D numpy array
        - iso_coords: (x, y) tuple
        - proximity: int
    """
    
    # Define the coordinates of the neighbors of the current isobox based on the proximity criteria
    x_coords = [max(iso_coords[0] - i, 0) for i in range(1, proximity + 1)] 
    x_coords.extend([min(iso_coords[0] + i, len(grid[0])) for i in range(1, proximity + 1)] + [iso_coords[0]])

    y_coords = [max(iso_coords[1] - i, 0) for i in range(1, proximity + 1)]
    y_coords.extend([min(iso_coords[1] + i, len(grid)) for i in range(1, proximity + 1)] + [iso_coords[1]])
    
    # Return the isobox numbers of the neighbors themselves
    return [iso_grid[x][y] for (x, y) in set(itertools.product(x_coords, y_coords)) if (x != iso_coords[0] or y != iso_coords[1])]

In [ ]:
# Assumption: people of same node properties interact if they are within close (variable=1) isobox proximity
iso_proximity = 2
isobox_x, isobox_y = 0, 0
isobox_neighbors = get_iso_neighbors(iso_grid, (isobox_x, isobox_y), iso_proximity)

In [ ]:
isobox_neighbors.sort()
isobox_neighbors

In [ ]:
def connect_with_neighbors(graph, iso_grid, iso_proximity, ethnicity):
    """
        TEMPORARY: Only connect nodes that share the same ethnicity
        TODO: Don't add edges if we have added them already (don't search for things twice?)
    """
    
    
    # For every possible isobox:
    for isobox in range(n_isoboxes):
        
        # Get all the nodes in the current isobox
        nodes = [node for node, attr in graph.nodes(data=True) if attr["isobox_num"] == isobox]
        
        # Get the coordinates of the isobox in the grid
        isobox_coords = (np.where(iso_grid == isobox)[0][0], np.where(iso_grid == isobox)[1][0])
        
        # Given its isobox number and coordinates in the iso grid, get its neighbor isoboxes
        isobox_neighbors = get_iso_neighbors(iso_grid, isobox_coords, iso_proximity)
        
        # For every neighbor isobox:
        for iso_neighbor in isobox_neighbors:
            
            # Get all the nodes in that isobox neighbor
            neighbor_nodes = [node for node, attr in graph.nodes(data=True) if attr["isobox_num"] == iso_neighbor]
            
            # If they share the same properties, draw an edge between them
            graph.add_edges_from([(node, neighbor_node) for node in nodes \
                                  for neighbor_node in neighbor_nodes if node["ethnicity"] == neighbor_node["ethnicity"]])
            
    